## 04. Graph coded frames

In [ ]:

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
sc.version

In [ ]:
scale = 1/200.0

codedFramesRootPath = "/user/fsainz/data/out/coded/"
#frame2graph = "2018/01/03/11/37/38-41858570"
frame2graph = "2018/01/03/11/41/56-42116913"
labeledFrames = "/user/fsainz/data/input/labeled"

filesPath = codedFramesRootPath + frame2graph

In [ ]:
print("Procesing: " + filesPath )

## Read labeled files and coded frames

In [ ]:
labelsRdd = sc.textFile(labeledFrames).map(lambda x: x.split(',')).map(lambda x: (int(x[0]), x[1]))
labelsRdd.count()


In [ ]:
labelsRdd.take(10)

In [ ]:
eventsStrRdd = sc.textFile(filesPath)

In [ ]:
eventsStrRdd.count
e = eventsStrRdd.first()

In [ ]:
def getFields( line ):
    line = line[1:-1]  # get rid of parenthesis
    line = line.split(';')
    #ret = ret.split(',')
    srcFrame =  list(map(  lambda x: float(x) , line[1].split(',')))
    codedFrame =list(map( lambda x: float(x) , line[2].split(',')))
    lossFrame = list(map( lambda x: float(x) , line[3].split(',')))
    ret = line[0].split(',')
    ret = ret[:-1]
    ret.extend([srcFrame,codedFrame,lossFrame])
    ret[1] = float(ret[1])
    ret[2] = int(float(ret[2]))
    ret[3] = float(ret[3])
    ret[0], ret[2] = ret[2],ret[0]
    return ret
    # return (ret, srcFrame, codedFrame, lossFrame)

In [ ]:
codedframesRdd = eventsStrRdd.map( lambda x: getFields(x)   ).map( lambda x: (x[0],x[1:]))

In [ ]:
codedlabeledRdd= codedframesRdd.join(labelsRdd).map( lambda x: (x[0], x[1][1], x[1][0][4],x[1][0][5],x[1][0][6] ))

In [ ]:
codedlabeledRdd.first()


## Trigger selecction (offline)

In [ ]:
labeledSamples = codedlabeledRdd.collect()
labeledSamples[0][4]

In [ ]:
n = len(labeledSamples)
print(n)

In [ ]:
def computeErrors( labeledSamples, trigger):
   
    LabelPos,LabelNeg,TotalError, FalsePos,FalseNeg = 0,0,0,0,0

    for item in labeledSamples:
        if item[1] != "N":    # "labeled as anomaly"
            LabelPos += 1
            if np.max(item[4]) < trigger : # Detected as anomaly
                TotalError += 1
                FalsePos += 1
        else: 
            LabelNeg +=1
            if np.max(item[4]) >= trigger:  # Labeled as Normal"
                TotalError += 1
                FalseNeg += 1


    return ( LabelPos, LabelNeg, trigger, TotalError, FalsePos, FalseNeg)


In [ ]:
computeErrors(labeledSamples, 0.15)

In [ ]:
trigger = 0.0
step = 0.01
steps = 100
TotalError= []
FalsePos  = []
FalseNeg  = []
Triggers  = []
for t in range(100):
    e = computeErrors( labeledSamples, trigger)
    Triggers.append(e[2])
    TotalError.append(e[3])
    FalsePos.append(e[4])
    FalseNeg.append(e[5])
    
    trigger += step

In [ ]:
x = Triggers
fig, ax = plt.subplots(2,1, sharex=True,figsize=(20,6))

ax[0].plot(x, TotalError,"green")
ax[1].plot(x, FalsePos, 'green', linewidth=1)
ax[1].plot(x, FalseNeg, 'green', linewidth=1)
display(fig)
fig.clear

In [ ]:
MinError = np.min(TotalError)
idx = TotalError.index(MinError)
t = Triggers[idx]
print("Min error", MinError/ n ,"@", t)


In [ ]:
trigger = 0.33
maxItemsDisplay = 20
l = len(labeledSamples[1][2])
x = range(00,l)


samples = codedlabeledRdd.takeSample(False, maxItemsDisplay, 0)
maxItemsDisplay
nsamples = min(codedframesRdd.count(), maxItemsDisplay)

fig, ax = plt.subplots(nsamples,3,figsize=(20,20), gridspec_kw = {'width_ratios':[3,3, 1]})
for idx, item in  enumerate( samples):
    if item[1] != "N":
        labelColor = "Red"
    else:
        labelColor = "green"
        
    if np.max(item[4]) > trigger:
        lossColor = "Red"
    else:
        lossColor = "green"
   
    ax[idx][0].plot(x, item[2], labelColor)
    ax[idx][0].plot(x, item[3], 'gray', linewidth=1)
    ax[idx][1].plot(x, item[4], lossColor, linewidth=1)
   # the histogram of the data
    j, bins, patches = ax[idx][2].hist( item[4], 50, normed=1, facecolor='gray')
                                                                                        
    
display(fig)
plt.close()


##### Graph coded frames RDD

In [ ]:
samples

In [ ]:
item=4
nsamples = codedframesRdd.count()
samples = codedframesRdd.take(10)


In [ ]:
samples[item][1][4]

In [ ]:
print(samples[item][0], np.max(samples[item][1][4]),  np.max(samples[item][1][5]),  np.max(samples[item][1][6] ))

In [ ]:

l = len(samples[item][1][5])
x = range(00,l)

fig, ax = plt.subplots(3,1, sharex=True,figsize=(20,6))

ax[0].plot(x, samples[item][1][4],"green")
ax[0].plot(x, samples[item][1][5], 'gray', linewidth=1)
ax[2].plot(x, samples[item][1][6], 'gray', linewidth=1)
display(fig)


In [ ]:
x1,x2 = 1,360
l = len(samples[item][1][5][x1:x2])
x = range(00,l)

fig, ax = plt.subplots(3,1, sharex=True,figsize=(20,6))

ax[0].plot(x, samples[item][1][4][x1:x2],"green")
ax[0].plot(x, samples[item][1][5][x1:x2], 'gray', linewidth=1)
ax[2].plot(x, samples[item][1][6][x1:x2], 'gray', linewidth=1)
display(fig)


In [ ]:
plt.close('all') 